In [1]:
import phrase_entity_extraction as ph

In [2]:
st = "101 SOUTH CENTER COURT".lower()
max_address = ph.return_max_address(ph.seq, st)
if st.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(st))
print("Max Address: ", max_address)
print()
print("Encoding: ", encoding)
print('\n\n{},"{}"'.format(st.upper(), encoding))

ALL GOOD!
Max Address:  101 SOUTH CENTER COURT

Encoding:  [['DIGIT', 'ADR_HEAD'], ['COMMONST', 'DIR'], ['WORDWAY', 'COMMONST'], ['WORDWAY']]


101 SOUTH CENTER COURT,"[['DIGIT', 'ADR_HEAD'], ['COMMONST', 'DIR'], ['WORDWAY', 'COMMONST'], ['WORDWAY']]"


In [3]:
arr_cands = ph.get_markers(ph.seq, st, ['ADDRESS', 'POBOX', 'SUITE'])
arr_cands

[[0, 3, 3, ['ADDRESS'], '101 south center'],
 [0, 4, 4, ['ADDRESS'], '101 south center court']]

In [4]:
ph.w(st)

['101', 'south', 'center', 'court']